<a href="https://colab.research.google.com/github/syym/ga-scheduling/blob/master/Schedule_ADSaitai_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install deap
!pip3 install scoop

In [0]:
import random
from scoop import futures

from deap import base
from deap import creator
from deap import tools
from deap import cma

In [0]:
# 従業員を表すクラス
class Employee(object):
  def __init__(self, no, name, priority, prevcount, wills, costs):
    self.no = no
    self.name = name
    self.priority = priority
    self.prevcount = prevcount
    self.wills = wills
    self.costs = costs

  def get_cost(self, box_index):
    return(self.costs[box_index])
    # return(self.costs[SHIFT_BOXES.index(box_name)])

  def is_applicated(self, box_name):
    return (box_name in self.wills)

In [0]:
  # インプットパラメータの定義
  DURATION_DAYS = 10
  DAYS_PER_PEOPLE = 2

  POPULATION_GROUP = 40


In [0]:
# シフトを表すクラス
# 内部的には 3(朝昼晩) * 7日 * 10人 = 210次元のタプルで構成される
class Shift(object):

  SHIFT_BOXES = []
  for i in range(DURATION_DAYS):
    SHIFT_BOXES.append('d' + str(i))

  # 各コマの想定人数
  NEED_PEOPLE = [DAYS_PER_PEOPLE] * DURATION_DAYS

  def __init__(self, list):
    if list == None:
      self.make_sample()
    else:
      self.list = list
    self.employees = []

  # ランダムなデータを生成
  def make_sample(self):
    sample_list = []
    for num in range(DURATION_DAYS * POPULATION_GROUP):
      sample_list.append(random.randint(0, 1))
    self.list = tuple(sample_list)

  # タプルを1ユーザ単位に分割
  def slice(self):
    sliced = []
    start = 0
    for num in range(POPULATION_GROUP):
      sliced.append(self.list[start:(start + DURATION_DAYS)])
      start = start + DURATION_DAYS
    return tuple(sliced)

  # ユーザ別にアサインコマ名を出力する
  def print_inspect(self):
    user_no = 0
    for line in self.slice():
      printf("ユーザ%d" % user_no)
      print(line)
      user_no = user_no + 1

      index = 0
      for e in line:
        if e == 1:
          print(self.SHIFT_BOXES[index])
        index = index + 1

  # CSV形式でアサイン結果の出力をする
  def print_csv(self):
    for line in self.slice():
      print(','.join(map(str, line)))

  # TSV形式でアサイン結果の出力をする
  def print_tsv(self):
    for line in self.slice():
      print("\t".join(map(str, line)))

  # ユーザ番号を指定してコマ名を取得する
  def get_boxes_by_user(self, user_no):
    line = self.slice()[user_no]
    return self.line_to_box(line)

  # 1ユーザ分のタプルからコマ名を取得する
  def line_to_box(self, line):
    result = []
    index = 0
    for e in line:
      if e == 1:
        result.append(self.SHIFT_BOXES[index])
      index = index + 1
    return result    

  # コマ番号を指定してアサインされているユーザ番号リストを取得する
  def get_user_nos_by_box_index(self, box_index):
    user_nos = []
    index = 0
    for line in self.slice():
      if line[box_index] == 1:
        user_nos.append(index)
      index += 1
    return user_nos

  # コマ名を指定してアサインされているユーザ番号リストを取得する
  def get_user_nos_by_box_name(self, box_name):
    box_index = self.SHIFT_BOXES.index(box_name)
    return self.get_user_nos_by_box_index(box_index)

  # 想定人数と実際の人数の差分を取得する
  def abs_people_between_need_and_actual(self):
    result = []
    index = 0
    for need in self.NEED_PEOPLE:
      actual = len(self.get_user_nos_by_box_index(index))
      result.append(abs(need - actual))
      index += 1
    return result

  # 調整コストの量を取得する
  def sum_total_cost(self):
    count = 0
    for box_name in self.SHIFT_BOXES:
      user_nos = self.get_user_nos_by_box_name(box_name)
      for user_no in user_nos:
        e = self.employees[user_no]
        count += e.get_cost(self.SHIFT_BOXES.index(box_name))
    return count

  # 過去の参加回数の総量を取得する
  def sum_previous_count(self):
    count = 0
    for box_name in self.SHIFT_BOXES:
      user_nos = self.get_user_nos_by_box_name(box_name)
      for user_no in user_nos:
        e = self.employees[user_no]
        count += e.prevcount
    return count

  # 優先度の総量を取得する
  def sum_priority(self):
    count = 0
    for box_name in self.SHIFT_BOXES:
      user_nos = self.get_user_nos_by_box_name(box_name)
      for user_no in user_nos:
        e = self.employees[user_no]
        count += e.priority
    return count

  # 応募していないコマにアサインされている件数を取得する
  def not_applicated_assign(self):
    count = 0
    for box_name in self.SHIFT_BOXES:
      user_nos = self.get_user_nos_by_box_name(box_name)
      for user_no in user_nos:
        e = self.employees[user_no]
        if not e.is_applicated(box_name):
          count += 1
    return count

  # 同一期間中に2回以上参加している人を取得する
  def multi_assigned_user(self):
    result = []
    for user_no in range(POPULATION_GROUP):
      if sum(self.slice()[user_no]) > 1:
        result.append(user_no)
    return len(result)


In [0]:
e0=Employee(0,'A1',3,1,['d1','d2','d6','d9'],[8,2,4,7,6,5,0,8,8,4])
e1=Employee(1,'B1',5,0,['d0','d1','d3','d4','d6','d9'],[4,0,8,1,3,7,4,7,5,4])
e2=Employee(2,'C1',0,2,['d0','d1','d2','d3','d4','d5','d6','d7','d8','d9'],[1,0,2,4,4,4,4,4,1,4])
e3=Employee(3,'D1',4,2,['d0','d3','d4','d6','d7','d9'],[1,5,8,1,1,8,2,4,5,0])
e4=Employee(4,'E1',1,2,['d1','d2','d3','d5','d6','d7','d8'],[7,4,1,1,7,3,3,3,3,5])
e5=Employee(5,'A2',5,2,['d7','d9'],[8,5,6,6,7,6,5,1,6,3])
e6=Employee(6,'B2',0,2,['d0','d1','d5','d6','d7','d8'],[3,1,5,8,7,2,4,3,1,5])
e7=Employee(7,'C2',3,2,['d0','d1','d3','d4','d7','d8'],[0,3,5,4,0,6,8,3,2,7])
e8=Employee(8,'D2',4,0,['d2','d5','d6','d7','d8','d9'],[7,5,1,6,6,2,1,4,2,2])
e9=Employee(9,'E2',3,1,['d1','d2','d3','d4','d5','d6','d8','d9'],[7,4,1,4,3,3,4,6,3,1])
e10=Employee(10,'A3',4,2,['d1','d2','d3','d4','d5','d6','d8','d9'],[5,1,3,4,2,2,3,6,3,2])
e11=Employee(11,'B3',2,1,['d0','d1','d2','d3','d4','d5','d6'],[1,3,3,2,3,3,4,6,6,5])
e12=Employee(12,'C3',2,2,['d0','d1','d2','d5','d6','d7','d8','d9'],[0,4,2,6,5,2,0,0,4,1])
e13=Employee(13,'D3',2,0,['d0','d2','d3','d4','d6','d9'],[1,5,3,3,3,7,1,7,7,0])
e14=Employee(14,'E3',3,2,['d2','d3','d4','d8'],[7,7,2,1,4,5,7,7,4,6])
e15=Employee(15,'A4',1,2,['d0','d1','d2','d4','d7'],[2,0,3,6,4,8,6,3,5,7])
e16=Employee(16,'B4',2,1,['d2','d3','d5','d7'],[8,8,2,4,7,1,7,4,5,5])
e17=Employee(17,'C4',1,1,['d0','d1','d3','d4','d6','d7','d8','d9'],[3,3,8,0,2,6,1,4,4,0])
e18=Employee(18,'D4',2,2,['d0','d2','d3','d4','d7','d8','d9'],[4,6,1,1,1,7,6,4,3,2])
e19=Employee(19,'E4',1,1,['d2','d3','d4','d5','d6','d7','d8','d9'],[6,5,1,0,2,1,4,1,4,3])
e20=Employee(20,'A5',4,1,['d0','d3','d4','d8'],[4,5,8,2,1,5,7,7,1,5])
e21=Employee(21,'B5',3,3,['d0','d1','d5','d6','d7','d8','d9'],[3,4,5,8,7,2,1,3,3,3])
e22=Employee(22,'C5',0,3,['d5','d9'],[5,7,7,8,5,3,5,7,6,3])
e23=Employee(23,'D5',3,2,['d1','d3','d5','d6','d8'],[7,3,6,2,5,3,2,5,4,7])
e24=Employee(24,'E5',4,1,['d1','d2','d5','d6','d8','d9'],[5,0,3,5,8,4,1,7,3,2])
e25=Employee(25,'A6',2,2,['d2','d4','d5','d6','d7'],[6,7,3,8,2,2,2,4,6,8])
e26=Employee(26,'B6',0,2,['d0','d2','d6','d7','d8'],[1,6,2,7,8,6,1,2,1,8])
e27=Employee(27,'C6',1,1,['d0','d2','d5','d6'],[4,6,2,6,7,2,0,6,7,5])
e28=Employee(28,'D6',4,1,['d2','d4','d5','d6','d7','d8','d9'],[7,7,4,8,1,1,1,3,4,3])
e29=Employee(29,'E6',4,3,['d0','d4','d7','d8','d9'],[3,7,5,6,3,6,7,4,3,3])
e30=Employee(30,'A7',0,0,['d0','d1','d2','d3','d4','d5','d6','d7'],[3,1,4,3,2,3,4,3,5,5])
e31=Employee(31,'B7',4,2,['d0','d3','d4','d6','d7','d8','d9'],[4,8,5,4,4,5,3,4,3,2])
e32=Employee(32,'C7',0,1,['d1','d3','d4','d9'],[7,2,5,2,2,7,6,7,7,4])
e33=Employee(33,'D7',3,1,['d0','d2','d3','d4','d7','d8'],[2,6,0,1,4,7,6,4,2,5])
e34=Employee(34,'E7',0,1,['d0','d4','d5','d6','d7','d8','d9'],[2,5,7,8,4,2,1,4,3,1])
e35=Employee(35,'A8',2,2,['d0','d1','d3','d4','d8'],[4,0,8,2,3,5,8,5,2,7])
e36=Employee(36,'B8',2,0,['d0','d1','d4','d9'],[1,2,5,5,3,5,5,8,5,1])
e37=Employee(37,'C8',4,3,['d0','d1','d2','d3','d4','d5','d6','d7','d8','d9'],[4,1,4,4,2,4,2,0,4,0])
e38=Employee(38,'D8',4,1,['d0','d2','d3','d4','d5','d6','d8'],[3,6,2,3,2,3,2,6,1,7])
e39=Employee(39,'E8',3,3,['d0','d5','d7','d8','d9'],[3,7,6,5,6,1,5,2,2,2])
employees = [e0, e1, e2, e3, e4, e5, e6, e7, e8, e9, e10, e11, e12, e13, e14, e15, e16, e17, e18, e19, e20, e21, e22, e23, e24, e25, e26, e27, e28, e29, e30, e31, e32, e33, e34, e35, e36, e37, e38, e39]

In [0]:
creator.create("FitnessPeopleCount", base.Fitness, weights=(-10.0, -100.0, -10.0, -10.0, -10.0, -50.0))
creator.create("Individual", list, fitness=creator.FitnessPeopleCount)

toolbox = base.Toolbox()

toolbox.register("map", futures.map)

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, (DURATION_DAYS * POPULATION_GROUP))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessPeopleCount' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [0]:
def evalShift(individual):
  s = Shift(individual)
  s.employees = employees

  # 想定人数とアサイン人数の差
  people_count_sub_sum = sum(s.abs_people_between_need_and_actual())
  # 応募していない時間帯へのアサイン数
  not_applicated_count = s.not_applicated_assign()
  # 調整コストの総量
  total_cost = s.sum_total_cost() / (POPULATION_GROUP)
  # 過去の参加回数の総量
  previous_count = s.sum_previous_count() / (POPULATION_GROUP)
  # 優先度の総量
  total_priority = s.sum_priority() / (POPULATION_GROUP)
  # 2回以上アサインされた人数
  people_count_multi_assign = s.multi_assigned_user()

  return (not_applicated_count, people_count_sub_sum, total_cost, previous_count, total_priority, people_count_multi_assign)


In [0]:
toolbox.register("evaluate", evalShift)
# 交叉関数を定義(二点交叉)
toolbox.register("mate", tools.cxTwoPoint)

# 変異関数を定義(ビット反転、変異隔離が5%)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# 選択関数を定義(トーナメント選択、tournsizeはトーナメントの数？)
toolbox.register("select", tools.selTournament, tournsize=3)

In [0]:
if __name__ == '__main__':
    # 初期集団を生成する
    pop = toolbox.population(n=300)
    CXPB, MUTPB, NGEN = 0.6, 0.5, 3000 # 交叉確率、突然変異確率、進化計算のループ回数

    print("進化開始")

    # 初期集団の個体を評価する
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):  # zipは複数変数の同時ループ
        # 適合性をセットする
        ind.fitness.values = fit

    print("  %i の個体を評価" % len(pop))

     # 進化計算開始
    for g in range(NGEN):
        print("-- %i 世代 --" % g)

        # 選択
        # 次世代の個体群を選択
        offspring = toolbox.select(pop, len(pop))
        # 個体群のクローンを生成
        offspring = list(map(toolbox.clone, offspring))

        # 選択した個体群に交差と突然変異を適応する

        # 交叉
        # 偶数番目と奇数番目の個体を取り出して交差
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                # 交叉された個体の適合度を削除する
                del child1.fitness.values
                del child2.fitness.values

        # 変異
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # 適合度が計算されていない個体を集めて適合度を計算
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  %i の個体を評価" % len(invalid_ind))

        # 次世代群をoffspringにする
        pop[:] = offspring

        # すべての個体の適合度を配列にする
        index = 1
        for v in ind.fitness.values:
          fits = [v for ind in pop]

          length = len(pop)
          mean = sum(fits) / length
          sum2 = sum(x*x for x in fits)
          std = abs(sum2 / length - mean**2)**0.5

          print("* パラメータ%d" % index)
          print("  Min %s" % min(fits))
          print("  Max %s" % max(fits))
          print("  Avg %s" % mean)
          print("  Std %s" % std)
          index += 1

    print("-- 進化終了 --")

    best_ind = tools.selBest(pop, 1)[0]
    print("最も優れていた個体: %s, %s" % (best_ind, best_ind.fitness.values))
    s = Shift(best_ind)
    s.print_csv()
    s.print_tsv()

進化開始
  300 の個体を評価
-- 0 世代 --
  231 の個体を評価
* パラメータ1
  Min 75.0
  Max 75.0
  Avg 75.0
  Std 0.0
* パラメータ2
  Min 166.0
  Max 166.0
  Avg 166.0
  Std 0.0
* パラメータ3
  Min 18.175
  Max 18.175
  Avg 18.1750000000001
  Std 1.5078914929239175e-06
* パラメータ4
  Min 7.2
  Max 7.2
  Avg 7.2000000000000215
  Std 4.2981520598697533e-07
* パラメータ5
  Min 11.35
  Max 11.35
  Avg 11.349999999999941
  Std 1.0254770835688861e-06
* パラメータ6
  Min 40.0
  Max 40.0
  Avg 40.0
  Std 0.0
-- 1 世代 --
  243 の個体を評価
* パラメータ1
  Min 74.0
  Max 74.0
  Avg 74.0
  Std 0.0
* パラメータ2
  Min 178.0
  Max 178.0
  Avg 178.0
  Std 0.0
* パラメータ3
  Min 18.975
  Max 18.975
  Avg 18.97500000000006
  Std 4.1295309247228556e-07
* パラメータ4
  Min 7.525
  Max 7.525
  Avg 7.525000000000042
  Std 6.307962682401158e-07
* パラメータ5
  Min 12.35
  Max 12.35
  Avg 12.34999999999994
  Std 8.920806376395086e-07
* パラメータ6
  Min 40.0
  Max 40.0
  Avg 40.0
  Std 0.0
-- 2 世代 --
  249 の個体を評価
* パラメータ1
  Min 65.0
  Max 65.0
  Avg 65.0
  Std 0.0
* パラメータ2
  Min 160.0
  Ma

In [0]:
print(s.abs_people_between_need_and_actual())
# print(s.not_applicated_assign() / (DURATION_DAYS * POPULATION_GROUP))
# print( s.sum_total_cost() / (DURATION_DAYS * POPULATION_GROUP))
# print(s.sum_previous_count() / (DURATION_DAYS * POPULATION_GROUP))
# print( s.sum_priority() / (DURATION_DAYS * POPULATION_GROUP))
print( s.multi_assigned_user())


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3


In [0]:
user_no = 0
ans = []
for i in s.slice():
  cnt = 0
  for j in i:
    if j == 1:
      tmp = []
      tmp.append("d" + str(cnt))
      tmp.append(employees[user_no].name)
      tmp.append(employees[user_no].prevcount)
      tmp.append(employees[user_no].priority)
      tmp.append(employees[user_no].costs[cnt])
      tmp.append(("d" + str(cnt)) in employees[user_no].wills)
      ans.append(tmp)
    cnt += 1
  user_no += 1
print(ans)

[['d2', 'E1', 2, 1, 1, True], ['d3', 'E1', 2, 1, 1, True], ['d1', 'B2', 2, 0, 1, True], ['d9', 'E2', 1, 3, 1, True], ['d8', 'E3', 2, 3, 4, True], ['d0', 'A4', 2, 1, 2, True], ['d9', 'C4', 1, 1, 0, True], ['d3', 'E4', 1, 1, 0, True], ['d5', 'E4', 1, 1, 1, True], ['d7', 'E4', 1, 1, 1, True], ['d5', 'B5', 3, 3, 2, True], ['d6', 'D5', 2, 3, 2, True], ['d8', 'D5', 2, 3, 4, True], ['d6', 'E5', 1, 4, 1, True], ['d7', 'E6', 3, 4, 4, True], ['d4', 'C7', 1, 0, 2, True], ['d2', 'D7', 1, 3, 0, True], ['d1', 'A8', 2, 2, 0, True], ['d0', 'B8', 0, 2, 1, True], ['d4', 'D8', 1, 4, 2, True]]
